In [1]:
import pandas as pd
import mysql.connector
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
import xgboost as xgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

%matplotlib inline

In [ ]:

# Connect to SQL Server
conn = mysql.connector.connect(host="localhost", user="root", password="root", database="GooglePlayStore")

# Fetch Data
query = "SELECT * FROM rating_pred"
df = pd.read_sql(query, conn)

# Close connection
conn.close()

df = df.drop('id', axis=1)
# Show first few rows
print(df.head())



C:\Users\shree\AppData\Local\Temp\ipykernel_17332\456756423.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   Category  Size_in_Mb  Content_Rating  Ad_Supported  In_App_Purchases  \
0         0        10.0               0             0                 0   
1         1         2.9               0             1                 0   
2         2         3.7               0             0                 0   
3         3         1.8               0             1                 0   
4         1         6.2               0             0                 0   

   Transformed_Rating  Installs  Free  Rating_Count  Editors_Choice  
0                   0        10     1             0               0  
1                   4      5000     1            64               0  
2                   0        50     1             0               0  
3                   5        10     1             5               0  
4                   0       100     1             0               0  


## Feature Extraction

In [3]:
df.shape

(2237013, 10)

In [4]:
import pandas as pd

# Assuming df is your DataFrame

# Filter out rows where Transformed_Rating is 0
zero_rating_rows = df[df["Transformed_Rating"] == 0]

# Randomly sample 500,000 rows from the filtered DataFrame
rows_to_drop = zero_rating_rows.sample(n=700000, random_state=42)

# Drop these rows from the original DataFrame
df = df.drop(rows_to_drop.index).reset_index(drop=True)

# Check the new shape
print("New DataFrame Shape:", df.shape)


New DataFrame Shape: (1537013, 10)


In [5]:
print(df.head())  
print(df.columns) 

   Category  Size_in_Mb  Content_Rating  Ad_Supported  In_App_Purchases  \
0         1         2.9               0             1                 0   
1         2         3.7               0             0                 0   
2         3         1.8               0             1                 0   
3         4        46.0               1             0                 1   
4         5         2.5               0             1                 0   

   Transformed_Rating  Installs  Free  Rating_Count  Editors_Choice  
0                   4      5000     1            64               0  
1                   0        50     1             0               0  
2                   5        10     1             5               0  
3                   0        50     1             0               0  
4                   5      1000     1            12               0  
Index(['Category', 'Size_in_Mb', 'Content_Rating', 'Ad_Supported',
       'In_App_Purchases', 'Transformed_Rating', 'Installs', '

In [6]:
from imblearn.combine import SMOTETomek
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

# Load dataset
X = df.drop("Transformed_Rating", axis=1)  # Features
Y = df["Transformed_Rating"]  # Target variable

# Original class distribution
print("🔹 Original Class Distribution:", Counter(Y))
# Check the new shape
print("New DataFrame Shape:", df.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3, random_state=7,stratify=Y)

# # Apply SMOTETomek (Combination)
smote_tomek = SMOTETomek(random_state=7)
X_train_resampled, Y_train_resampled = smote_tomek.fit_resample(X_train, Y_train)
print("🔹 After SMOTETomek:", Counter(Y_train_resampled))

# # Choose the final resampled data (you can pick any of the resampled data)
# X = X_combined  
# Y = y_combined  

# # Ensure X and y have the same number of samples after resampling
print(f"X shape: {X_train_resampled.shape}, Y shape: {Y_train_resampled.shape}")



🔹 Original Class Distribution: Counter({4: 561859, 5: 412277, 0: 346271, 3: 173693, 2: 38770, 1: 4143})
New DataFrame Shape: (1537013, 10)
🔹 After SMOTETomek: Counter({0: 393291, 1: 388089, 2: 376311, 3: 364145, 5: 353763, 4: 343229})
X shape: (2218828, 9), Y shape: (2218828,)


In [ ]:
# Feature and Target
# X = df.drop('Transformed_Rating', axis=1)
# Y = df['Transformed_Rating'].astype(int)  # Ensure categorical target

# # Split data
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=7, stratify=Y)
print("Shape of X_train:", X_train.shape)
print("Shape of Y_train:", Y_train.shape)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgbc = HistGradientBoostingClassifier(max_iter=100, learning_rate=0.1, random_state=7)
hgbc.fit(X_train, Y_train)
y_pred = hgbc.predict(X_test)
hgbc_ac = accuracy_score(Y_test, y_pred)

print("HistGradientBoosting Classifier Accuracy:", hgbc_ac)

In [ ]:
#Applying Model Logistic Regression

logreg_c=LogisticRegression(max_iter=500, random_state=7, class_weight='balanced')
logreg_c.fit(X_train,Y_train)
logreg_pred=logreg_c.predict(X_test)
logreg_cm=confusion_matrix(Y_test,logreg_pred)
logreg_ac=accuracy_score(Y_test, logreg_pred)
print('LogisticRegression_accuracy:',logreg_ac)

In [ ]:
#Applying Model RandomForest

rdf_c=RandomForestClassifier(random_state=7)
rdf_c.fit(X_train,Y_train)
rdf_pred=rdf_c.predict(X_test)
rdf_cm=confusion_matrix(Y_test,rdf_pred)
rdf_ac=accuracy_score(rdf_pred,Y_test)
print('RandomForest_Accuracy: ', rdf_ac)

In [ ]:
#Applying Model DecisionTree Classifier

dtree_c=DecisionTreeClassifier(random_state=7,criterion='entropy', max_depth = 10, min_samples_leaf = 2, min_samples_split = 5)
dtree_c.fit(X_train,Y_train)
dtree_pred=dtree_c.predict(X_test)
dtree_cm=confusion_matrix(Y_test,dtree_pred)
dtree_ac=accuracy_score(dtree_pred,Y_test)
print('DecisionTreeClassifier_Accuracy: ',dtree_ac)

with open("rating_model.pkl", "wb") as f:
    pickle.dump(dtree_c, f)

In [ ]:
#Applying Model Naive Bayesian

NB = BernoulliNB(binarize = 0.0)
NB.fit(X_train,Y_train)
y_pred = NB.predict(X_test)
nb_ac=accuracy_score(Y_test, y_pred)
print("Bernoulli Naive Bayes_Accuracy: ", nb_ac)

In [ ]:
# Applying AdaBoost Classifier
abc = AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=7)
abc.fit(X_train, Y_train)
y_pred = abc.predict(X_test)
abc_ac = accuracy_score(Y_test, y_pred)
print("AdaBoost Classifier Accuracy:", abc_ac)

In [ ]:
# Applying XGBoost Classifier
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=7, use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, Y_train)
y_pred = xgb.predict(X_test)
xgb_ac = accuracy_score(Y_test, y_pred)
print("XGBoost Classifier Accuracy:", xgb_ac)

In [ ]:
# Initialize LightGBM classifier
lgb_model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=7)
lgb_model.fit(X_train, Y_train)
y_pred = lgb_model.predict(X_test)
lgb_ac = accuracy_score(Y_test, y_pred)
print("LightGBM Classifier Accuracy:", lgb_ac)

In [ ]:
# Fine-tuning CatBoost Classifier
param_grid_catboost = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8]
}

catboost = CatBoostClassifier(verbose=0)
grid_search_catboost = GridSearchCV(catboost, param_grid_catboost, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_catboost.fit(X_train, Y_train)

# Best parameters and score for CatBoost
print("Best Parameters (CatBoost):", grid_search_catboost.best_params_)
print("Best Accuracy (CatBoost):", grid_search_catboost.best_score_)

In [ ]:
# # Applying Model CatBoost Model

Cat_Boost = CatBoostClassifier(verbose = 300, n_estimators = 3500, learning_rate=0.1, depth=10,early_stopping_rounds=300)
# Cat_Boost = CatBoostClassifier(verbose = 0)
Cat_Boost.fit(X_train, Y_train)
cb_ac=Cat_Boost.score(X_train, Y_train)
print("CatBoost_Accuracy: ",cb_ac)

with open("rating_model_one.pkl", "wb") as f:
    pickle.dump(Cat_Boost, f)

In [15]:
from sklearn.metrics import f1_score

#Applying Model CatBoost Model

Cat_Boost = CatBoostClassifier(verbose = 300, n_estimators=1000, learning_rate=0.2, depth=10,early_stopping_rounds=300)
# Cat_Boost = CatBoostClassifier(verbose = 0)
Cat_Boost.fit(X_train_resampled, Y_train_resampled)

# Model Accuracy
cb_ac = Cat_Boost.score(X_train_resampled, Y_train_resampled)
print("CatBoost Accuracy: ", cb_ac)

# Predictions
y_pred = Cat_Boost.predict(X_test)

# F1-Score Calculation
f1 = f1_score(Y_test, y_pred, average="weighted")  # Use 'weighted' for multi-class
print("F1 Score:", f1)

with open("finetuning_three.pkl", "wb") as f:
    pickle.dump(Cat_Boost, f)

0:	learn: 1.5333701	total: 3.59s	remaining: 59m 44s
300:	learn: 0.9152899	total: 19m 51s	remaining: 46m 6s
600:	learn: 0.8552416	total: 39m	remaining: 25m 54s
900:	learn: 0.8172602	total: 58m 35s	remaining: 6m 26s
999:	learn: 0.8072548	total: 1h 4m 48s	remaining: 0us
CatBoost Accuracy:  0.6752871335678114
F1 Score: 0.6254207514947179


In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, roc_curve, auc
import matplotlib.pyplot as plt
import pickle

# Train CatBoost model
Cat_Boost = CatBoostClassifier(verbose=0)
Cat_Boost.fit(X_train, Y_train)

# Model Accuracy
cb_ac = Cat_Boost.score(X_train, Y_train)
print("CatBoost Accuracy: ", cb_ac)

# Predictions
y_pred = Cat_Boost.predict(X_test)

# **F1-Score Calculation**
f1 = f1_score(Y_test, y_pred, average="weighted")  # Use 'weighted' for multi-class
print("F1 Score:", f1)

# **ROC Curve Calculation**
y_prob = Cat_Boost.predict_proba(X_test)  # Get probability estimates
if len(set(Y_test)) == 2:  # Binary classification
    fpr, tpr, _ = roc_curve(Y_test, y_prob[:, 1])  # Only second column for positive class
    roc_auc = auc(fpr, tpr)

    # Plot ROC Curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color="blue", lw=2, label=f"ROC curve (area = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], color="gray", linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve - CatBoost")
    plt.legend(loc="lower right")
    plt.show()
    print("ROC AUC Score:", roc_auc)
else:
    print("ROC curve is not applicable for multi-class classification.")

# Save the model
with open("rating_model.pkl", "wb") as f:
    pickle.dump(Cat_Boost, f)
